## 模型参数的访问，初始化，共享

In [13]:
import torch
from torch import nn
from torch.nn import init

net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1))  # pytorch已进行默认初始化

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


##### 访问模型参数

In [14]:
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name, param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [15]:
#单层
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


##### 初始化模型参数

In [16]:
#我们将权重参数初始化成均值为0、标准差为0.01的正态分布随机数，并依然将偏差参数清零。
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

0.weight tensor([[ 0.0041, -0.0091,  0.0044, -0.0036],
        [-0.0080, -0.0192,  0.0129, -0.0168],
        [ 0.0023,  0.0064,  0.0065,  0.0137]])
2.weight tensor([[0.0107, 0.0181, 0.0023]])


In [17]:
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)


0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


##### 自定义初始化方法

In [18]:
#pytorch中怎么初始化的
def normal_(tensor, mean=0, std=1):
    with torch.no_grad():
        return tensor.normal_(mean, std)

In [19]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float()

for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

0.weight tensor([[-0.0000, -0.0000, -0.0000, -0.0000],
        [-0.0000,  9.2466, -6.7735,  0.0000],
        [-6.1619, -0.0000, -0.0000,  8.0185]])
2.weight tensor([[ 6.3666, -6.0612,  9.3993]])


##### 共享模型参数

In [20]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear) 
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)


Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [21]:
#两个线性层其实是一个
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))


True
True


In [22]:
x = torch.ones(1,1)
y=net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad)# 单次梯度是3，两次所以就是6

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
